In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import libraries - PyTorch

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
import seaborn as sns
from sklearn.metrics import confusion_matrix
from tempfile import TemporaryDirectory
import time

import torch
import torch.nn as nn
from torch import Tensor
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.backends.cudnn as cudnn

import torchvision
from torchvision.models import resnet101, resnet50, resnet18
import torchvision.transforms as transforms
import torchvision.transforms as tt
from torchvision.datasets import CIFAR100 
from torchvision import datasets, models, transforms
from torchinfo import summary

from tqdm import tqdm

In [ ]:
torch.manual_seed(0)

In [ ]:
!rm -rf /kaggle/working/*
!rm -rf /kaggle/working/models/*

In [ ]:
os.makedirs("/kaggle/working/models",exist_ok=True)

In [ ]:
os.makedirs("/kaggle/working/aftermodels",exist_ok=True)

# Compute stats

In [ ]:
stat_train_data = CIFAR100(download=True, root="./data", transform=tt.ToTensor())
stat_test_data = CIFAR100(root="./data", train=False, transform=tt.ToTensor())

In [ ]:
imgs_train = torch.stack([img_t for img_t ,_ in stat_train_data],dim=3)
imgs_test = torch.stack([img_t for img_t ,_ in stat_test_data],dim=3)
imgs = torch.cat((imgs_train, imgs_test), 3)
mean = imgs.view(3,-1).mean(dim=1).numpy()
std = imgs.view(3, -1).std(dim=1).numpy()
print(mean, std)

# Stat and transform

In [ ]:
stats=((0.50736207, 0.4866896, 0.44108862), (0.26748815, 0.2565931, 0.2763085))
train_transform = tt.Compose([
    tt.RandomHorizontalFlip(),
    tt.RandomCrop(32, padding=4, padding_mode="reflect"),
    tt.ToTensor(),
    tt.Normalize(*stats)
])

test_transform = tt.Compose([
    tt.ToTensor(),
    tt.Normalize(*stats)
])

# Import dataset 100 classes

In [ ]:
train_data = CIFAR100(download=True, root="./data", transform=train_transform)
test_data = CIFAR100(root="./data", train=False, transform=test_transform)

In [ ]:
batch_size=128
num_work=4
train_dl = DataLoader(train_data, batch_size, shuffle=True, num_workers=num_work, pin_memory=True)
test_dl = DataLoader(test_data, batch_size, shuffle=True, num_workers=num_work, pin_memory=True)

# Import dataset 20 classes

In [ ]:
class CIFAR100Coarse(torch.utils.data.Dataset):
    def __init__(self, cifar100_dataset) ->None:
        self.cifar100_dataset = cifar100_dataset
        
        # Define the mapping from fine labels to coarse labels
        self.fine_to_coarse={
            0: 4, 1: 1, 2: 14, 3: 8, 4: 0, 5: 6, 6: 7, 7: 7, 8: 18, 9: 3,
            10: 3, 11: 14, 12: 9, 13: 18, 14: 7, 15: 11, 16: 3, 17: 9, 18: 7,
            19: 11, 20: 6, 21: 11, 22: 5, 23: 10, 24: 7, 25: 6, 26: 13, 27: 15,
            28: 3, 29: 15, 30: 0, 31: 11, 32: 1, 33: 10, 34: 12, 35: 14, 36: 16,
            37: 9, 38: 11, 39: 5, 40: 5, 41: 19, 42: 8, 43: 8, 44: 15, 45: 13,
            46: 14, 47: 17, 48: 18, 49: 10, 50: 16, 51: 4, 52: 17, 53: 4, 54: 2,
            55: 0, 56: 17, 57: 4, 58: 18, 59: 17, 60: 10, 61: 3, 62: 2, 63: 12,
            64: 12, 65: 16, 66: 12, 67: 1, 68: 9, 69: 19, 70: 2, 71: 10, 72: 0,
            73: 1, 74: 16, 75: 12, 76: 9, 77: 13, 78: 15, 79: 13, 80: 16, 81: 19,
            82: 2, 83: 4, 84: 6, 85: 19, 86: 5, 87: 5, 88: 8, 89: 19, 90: 18, 91: 1,
            92: 2, 93: 15, 94: 6, 95: 0, 96: 17, 97: 8, 98: 14, 99: 13
        }
        
        # Mapping from superclass labels to superclass names
        self.coarse_label_names = {
            0: 'aquatic mammals',
            1: 'fish',
            2: 'flowers',
            3: 'food containers',
            4: 'fruit and vegetables',
            5: 'household electrical devices',
            6: 'household furniture',
            7: 'insects',
            8: 'large carnivores',
            9: 'large man-made outdoor things',
            10: 'large natural outdoor scenes',
            11: 'large omnivores and herbivores',
            12: 'medium-sized mammals',
            13: 'non-insect invertebrates',
            14: 'people',
            15: 'reptiles',
            16: 'small mammals',
            17: 'trees',
            18: 'vehicles 1',
            19: 'vehicles 2'
        }

    def __getitem__(self, index):
        img, target = self.cifar100_dataset[index]

        # Convert fine label to coarse label
        target = self.fine_to_coarse[target]

        return img, target

    def __len__(self):
        return len(self.cifar100_dataset)

    def get_coarse_label_name(self, label):
        return self.coarse_label_names[label]

    def get_all_coarse_label_names(self):
        return self.coarse_label_names.values()


In [ ]:
train_data_20=CIFAR100Coarse(train_data)
test_data_20=CIFAR100Coarse(test_data)

In [ ]:
batch_size=128
num_work=4
train_dl_20 = DataLoader(train_data_20, batch_size, shuffle=True, num_workers=num_work, pin_memory=True)
test_dl_20 = DataLoader(test_data_20, batch_size, shuffle=True, num_workers=num_work, pin_memory=True)

## Print number of items for each superclass

In [ ]:
train_classes_20={}

for img,label in train_data_20:
    class_name=train_data_20.coarse_label_names[label]
    if class_name not in train_classes_20:
        train_classes_20[class_name]=1
    else:
        train_classes_20[class_name]+=1

In [ ]:
train_classes_20

# Device - GPU cuda

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Model

In [ ]:
def make_model(num_classes=20):
    model=resnet18(weights=None)
    model.conv1=nn.Conv2d(3,64,kernel_size=3,stride=1,padding=1,bias=False)
    model.maxpool = nn.Identity()
    num_in_ftrs=model.fc.in_features
    num_out_ftrs=model.fc.out_features
    model.fc=nn.Linear(num_in_ftrs,num_classes)
    return model

In [ ]:
model=resnet18(weights=None)
model.conv1=nn.Conv2d(3,64,kernel_size=3,stride=1,padding=1,bias=False)
model.maxpool = nn.Identity()
num_in_ftrs=model.fc.in_features
num_out_ftrs=model.fc.out_features

In [ ]:
num_classes=20
model.fc=nn.Linear(num_in_ftrs,num_classes)

In [ ]:
num_classes=100
model.fc=nn.Linear(num_in_ftrs,num_classes)

In [ ]:
model=model.to(device)

In [ ]:
model

In [ ]:
summary(model,(128,3,32,32))

# Optimizer, scheduler, criterion

In [ ]:
# For optimizer:
max_lr=5e-2
momentum=0.9
weight_decay = 5e-4
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=max_lr, momentum=momentum, weight_decay=weight_decay)

In [ ]:
 scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[5, 10, 15, 25], gamma=0.1)

In [ ]:
 scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[40,55,60,80,120,140], gamma=0.1)

In [ ]:
 scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50,60,80,120,140], gamma=0.2)

In [ ]:
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=7, min_lr=5e-05)

In [ ]:
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.3)

# Training and validation loop

In [ ]:
num_epochs=300

In [ ]:
!rm -rf /kaggle/working/models/*

# Train for 20 classes

In [ ]:
train_loss=[]
train_acc=[]
test_loss=[]
test_acc=[]
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    data_loader = tqdm(train_dl_20, total=len(train_dl_20), desc=f'Epoch [{epoch + 1}/{num_epochs}]')

    for batch_idx, (inputs, targets) in enumerate(data_loader):
        optimizer.zero_grad()
        inputs, targets = inputs.to(device), targets.to(device)


        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        _, predicted = outputs.max(1)
        correct_predictions += predicted.eq(targets).sum().item()
        total_samples += targets.size(0)

        # Progress bar description
        data_loader.set_postfix(loss=total_loss / (batch_idx + 1), accuracy=correct_predictions / total_samples)

    average_loss = total_loss / len(train_dl_20)
    accuracy = correct_predictions / total_samples
    
    train_loss.append(average_loss)
    train_acc.append(accuracy)

    # test the model
    correct_test, total_test, total_loss_test = 0, 0, 0
    with torch.no_grad():
        for (inputs, targets) in test_dl_20:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss_test += loss.item()

            __, predicted = outputs.max(1)
            correct_test += predicted.eq(targets).sum().item()
            total_test+= targets.size(0)

        accuracy_test = correct_test / total_test
        average_loss_test = total_loss_test / len(test_dl_20)

    print(f'Epoch [{epoch + 1}/{num_epochs}] - Loss: {average_loss:.4f}, Accuracy: {accuracy * 100:.2f}%, Loss on test data: {average_loss_test:.4f} Accuracy on test data: {100 * accuracy_test:.2f} %')
    
    test_loss.append(average_loss_test)
    test_acc.append(accuracy_test)

    model_save_path = '/kaggle/working/models/cifar100_20classes_ep' + str(epoch+1) + '.pth'
    training_stats = {
      'epoch': epoch+1,
      'model_state_dict': model.state_dict(),
      'optimizer_state_dict': optimizer.state_dict(),
      'scheduler_state_dict': scheduler.state_dict(),
      'loss': average_loss,
      'accuracy': accuracy,
      'test_accuracy': accuracy_test,
      'test_loss': average_loss_test,
      'learning_rate': optimizer.param_groups[0]["lr"]
    }
    
#     if epoch%5==0:
#         model_save_path = '/kaggle/working/models/resnet_cifar10_' + str(epoch+1) + '.pth'
#         training_stats = {
#           'epoch': epoch+1,
#           'model_state_dict': model.state_dict(),
#           'optimizer_state_dict': optimizer.state_dict(),
#           'scheduler_state_dict': scheduler.state_dict(),
#           'loss': average_loss,
#           'accuracy': accuracy,
#           'test_accuracy': accuracy_test,
#           'test_loss': average_loss_test,
#           'learning_rate': optimizer.param_groups[0]["lr"]
#         }
    scheduler.step()
    torch.save(training_stats, model_save_path)

## Accuracy and loss plots

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Dokładność w procesie uczenia na zbiorze treningowym i testowym")
plt.plot(test_acc[:101],label="Zbiór testowy")
plt.plot(train_acc[:101],label="Zbiór treningowy")
plt.xlabel("Epoka")
plt.ylabel("Dokładność")
plt.xticks(np.arange(0,101,10))
plt.yticks(np.arange(0.0,1.1,0.1))
plt.grid()
plt.legend()
plt.savefig('cifar100_100classes_accuracyplot.png')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Dokładność w procesie uczenia na zbiorze treningowym i testowym")
plt.plot(test_acc[:151],label="Zbiór testowy")
plt.plot(train_acc[:151],label="Zbiór treningowy")
plt.xlabel("Epoka")
plt.ylabel("Dokładność")
plt.xticks(np.arange(0,151,10))
plt.grid()
plt.legend()
plt.savefig('cifar100_20classes_accuracyplot_I.png')
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Wartość straty w procesie uczenia na zbiorze treningowym i testowym")
plt.plot(test_loss[:101],label="Zbiór testowy")
plt.plot(train_loss[:101],label="Zbiór treningowy")
plt.xlabel("Epoka")
plt.ylabel("Strata")
plt.xticks(np.arange(0,101,10))
plt.grid()
plt.legend()
plt.savefig('cifar100_100classes_lossplot.png')
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Wartość straty w procesie uczenia na zbiorze treningowym i testowym")
plt.plot(test_loss[:151],label="Zbiór testowy")
plt.plot(train_loss[:151],label="Zbiór treningowy")
plt.xlabel("Epoka")
plt.ylabel("Strata")
plt.xticks(np.arange(0,151,10))
plt.grid()
plt.legend()
plt.savefig('cifar100_20classes_lossplot_I.png')
plt.show()

## Confusion matrix

In [ ]:
model.eval()
correct_test, total_test= 0, 0
all_preds = []
all_labels = []
with torch.no_grad():
    for (inputs, targets) in test_dl:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        __, predicted = outputs.max(1)
        correct_test += predicted.eq(targets).sum().item()
        total_test+= targets.size(0)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(targets.cpu().numpy())

accuracy_test = correct_test / total_test
accuracy_test_prc=accuracy_test*100
print(f'Accuracy on test data: {accuracy_test_prc:.2f} %')

In [ ]:
all_preds_coarse=[]
for i in range(len(all_preds)):
    all_preds_coarse.append(train_data_20.fine_to_coarse[all_preds[i]])

all_labels_coarse=[]
for i in range(len(all_preds)):
    all_labels_coarse.append(train_data_20.fine_to_coarse[all_labels[i]])
    
cm=confusion_matrix(all_labels_coarse,all_preds_coarse)

In [ ]:
cm = confusion_matrix(all_labels, all_preds)

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', linecolor='black', linewidth=0.5)
plt.xlabel('Klasa przewidywana')
plt.ylabel('Klasa rzeczywista ')
plt.title('Macierz pomyłek')
plt.savefig('CM_100klas.png')
plt.show()

# Train for 100 classes

In [ ]:
train_loss=[]
train_acc=[]
test_loss=[]
test_acc=[]
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    data_loader = tqdm(train_dl, total=len(train_dl), desc=f'Epoch [{epoch + 1}/{num_epochs}]')

    for batch_idx, (inputs, targets) in enumerate(data_loader):
        optimizer.zero_grad()
        inputs, targets = inputs.to(device), targets.to(device)


        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        _, predicted = outputs.max(1)
        correct_predictions += predicted.eq(targets).sum().item()
        total_samples += targets.size(0)

        # Progress bar description
        data_loader.set_postfix(loss=total_loss / (batch_idx + 1), accuracy=correct_predictions / total_samples)

    average_loss = total_loss / len(train_dl)
    accuracy = correct_predictions / total_samples

    train_loss.append(average_loss)
    train_acc.append(accuracy)
    
    # test the model
    correct_test, total_test, total_loss_test = 0, 0, 0
    with torch.no_grad():
        for (inputs, targets) in test_dl:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss_test += loss.item()

            __, predicted = outputs.max(1)
            correct_test += predicted.eq(targets).sum().item()
            total_test+= targets.size(0)

        accuracy_test = correct_test / total_test
        average_loss_test = total_loss_test / len(test_dl)
   
    test_loss.append(average_loss_test)
    test_acc.append(accuracy_test)

    print(f'Epoch [{epoch + 1}/{num_epochs}] - Loss: {average_loss:.4f}, Accuracy: {accuracy * 100:.2f}%, Loss on test data: {average_loss_test:.4f} Accuracy on test data: {100 * accuracy_test:.2f} %')


#     if epoch%5==0:
#         model_save_path = '/kaggle/working/resnet_cifar100_100classes_e' + str(epoch+1) + '.pth'
#         training_stats = {
#           'epoch': epoch+1,
#           'model_state_dict': model.state_dict(),
#           'optimizer_state_dict': optimizer.state_dict(),
#           'scheduler_state_dict': scheduler.state_dict(),
#           'loss': average_loss,
#           'accuracy': accuracy,
#           'test_accuracy': accuracy_test,
#           'test_loss': average_loss_test,
#           'learning_rate': optimizer.param_groups[0]["lr"]
#         }
    model_save_path = '/kaggle/working/resnet_cifar100_100classes_e' + str(epoch+1) + '.pth'
    training_stats = {
      'epoch': epoch+1,
      'model_state_dict': model.state_dict(),
      'optimizer_state_dict': optimizer.state_dict(),
      'scheduler_state_dict': scheduler.state_dict(),
      'loss': average_loss,
      'accuracy': accuracy,
      'test_accuracy': accuracy_test,
      'test_loss': average_loss_test,
      'learning_rate': optimizer.param_groups[0]["lr"]
    }
    scheduler.step()
    torch.save(training_stats, model_save_path)

In [ ]:
train_loss=[]
train_acc=[]
test_loss=[]
test_acc=[]
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    data_loader = tqdm(train_dl, total=len(train_dl), desc=f'Epoch [{epoch + 1}/{num_epochs}]')

    for batch_idx, (inputs, targets) in enumerate(data_loader):
        optimizer.zero_grad()
        inputs, targets = inputs.to(device), targets.to(device)


        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        _, predicted = outputs.max(1)
        correct_predictions += predicted.eq(targets).sum().item()
        total_samples += targets.size(0)

        # Progress bar description
        data_loader.set_postfix(loss=total_loss / (batch_idx + 1), accuracy=correct_predictions / total_samples)

    average_loss = total_loss / len(train_dl)
    accuracy = correct_predictions / total_samples

    train_loss.append(average_loss)
    train_acc.append(accuracy)
    
    # test the model
    correct_test, total_test, total_loss_test = 0, 0, 0
    with torch.no_grad():
        for (inputs, targets) in test_dl:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss_test += loss.item()

            __, predicted = outputs.max(1)
            correct_test += predicted.eq(targets).sum().item()
            total_test+= targets.size(0)

        accuracy_test = correct_test / total_test
        average_loss_test = total_loss_test / len(test_dl)
   
    test_loss.append(average_loss_test)
    test_acc.append(accuracy_test)

    print(f'Epoch [{epoch + 1}/{num_epochs}] - Loss: {average_loss:.4f}, Accuracy: {accuracy * 100:.2f}%, Loss on test data: {average_loss_test:.4f} Accuracy on test data: {100 * accuracy_test:.2f} %')


#     if epoch%5==0:
#         model_save_path = '/kaggle/working/resnet_cifar100_100classes_e' + str(epoch+1) + '.pth'
#         training_stats = {
#           'epoch': epoch+1,
#           'model_state_dict': model.state_dict(),
#           'optimizer_state_dict': optimizer.state_dict(),
#           'scheduler_state_dict': scheduler.state_dict(),
#           'loss': average_loss,
#           'accuracy': accuracy,
#           'test_accuracy': accuracy_test,
#           'test_loss': average_loss_test,
#           'learning_rate': optimizer.param_groups[0]["lr"]
#         }
    model_save_path = '/kaggle/working/resnet_cifar100_100classes_e' + str(epoch+1) + '.pth'
    training_stats = {
      'epoch': epoch+1,
      'model_state_dict': model.state_dict(),
      'optimizer_state_dict': optimizer.state_dict(),
      'scheduler_state_dict': scheduler.state_dict(),
      'loss': average_loss,
      'accuracy': accuracy,
      'test_accuracy': accuracy_test,
      'test_loss': average_loss_test,
      'learning_rate': optimizer.param_groups[0]["lr"]
    }
    scheduler.step()
    torch.save(training_stats, model_save_path)

# Plots

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Training and Testing Accuracy")
plt.plot(test_acc,label="Testing accuracy")
plt.plot(train_acc,label="Training accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.xticks(np.arange(0,121,10))
plt.yticks(np.arange(0.0,1.1,0.05))
plt.grid()
plt.legend()
plt.savefig('cifar100_100classes_accuracyplot_new03')
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Training and Testing Loss")
plt.plot(test_loss,label="Testing loss")
plt.plot(train_loss,label="Training loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.xticks(np.arange(0,121,10))
plt.grid()
plt.legend()
plt.savefig('cifar100_100classes_lossplot_new02')
plt.show()

## Saving checkpoint separately

In [ ]:
model_save_path = '/kaggle/working/resnet_cifar100_' + str(87) + '.pth'
training_stats = {
  'epoch': epoch+1,
  'model_state_dict': model.state_dict(),
  'optimizer_state_dict': optimizer.state_dict(),
  'scheduler_state_dict': scheduler.state_dict(),
  'loss': average_loss,
  'accuracy': accuracy,
  'test_accuracy': accuracy_test,
  'test_loss': average_loss_test,
  'learning_rate': optimizer.param_groups[0]["lr"]
}

In [ ]:
torch.save(training_stats, model_save_path)

# Retraining model for 100 classes

## Model setup

In [ ]:
saved_model_path='/kaggle/input/model-01/cifar100_20classes_ep53.pth'

In [ ]:
saved_model_path='/kaggle/input/pretrained-model/resnet_cifar100_transferlearningfor100clasep1415.pth'

In [ ]:
saved_model_path = '/kaggle/working/resnet_cifar100_87_20cl.pth'

In [ ]:
saved_model_path='/kaggle/input/premodel/resnet_cifar100_87_20cl.pth'

In [ ]:
saved_model_path='/kaggle/input/model25/resnet_cifar100_20classes_e25.pth'

In [ ]:
saved_model_path='/kaggle/working/models/resnet_cifar10_53.pth'

In [ ]:
model_test = resnet18(weights=None)

In [ ]:
model_test.conv1=nn.Conv2d(3,64,kernel_size=3,stride=1,padding=1,bias=False)
model_test.maxpool=nn.Identity()
num_classes = 20
test_in_ftrs=model_test.fc.in_features
test_out_ftrs=model_test.fc.out_features
model_test.fc=nn.Linear(test_in_ftrs,num_classes)

## Model weights loading

In [ ]:
checkpoint = torch.load(saved_model_path)
model_test.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
for param in model_test.parameters():
    param.requires_grad = False

In [ ]:
for param in model_test.fc.parameters():
    param.requires_grad = True

In [ ]:
for name, param in model_test.named_parameters():
    print(name)

In [ ]:
for param in model_test.parameters():
    print(param.requires_grad)

In [ ]:
num_classes=100
model_test.fc = nn.Linear(test_in_ftrs, num_classes)

In [ ]:
model_test.to(device)

In [ ]:
model_test.eval().to(device)

### Evaluation for 20 classes

In [ ]:
model_test.eval()
correct_test, total_test= 0, 0
with torch.no_grad():
    for (inputs, targets) in test_dl_20:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model_test(inputs)
        __, predicted = outputs.max(1)
        correct_test += predicted.eq(targets).sum().item()
        total_test+= targets.size(0)

accuracy_test = correct_test / total_test
accuracy_test_prc=accuracy_test*100
print(f'Accuracy on test data: {accuracy_test_prc:.2f} %')

### Evaluation for 100 classes

In [ ]:
model_test.eval()
correct_test, total_test= 0, 0
with torch.no_grad():
    for (inputs, targets) in test_dl:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model_test(inputs)
        __, predicted = outputs.max(1)
        correct_test += predicted.eq(targets).sum().item()
        total_test+= targets.size(0)

accuracy_test = correct_test / total_test
accuracy_test_prc=accuracy_test*100
print(f'Accuracy on test data: {accuracy_test_prc:.2f} %')

## Optimizer, scheduler etc.

In [ ]:
max_lr=5e-2
momentum=0.9
weight_decay = 5e-4
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_test.parameters(), lr=max_lr, momentum=momentum, weight_decay=weight_decay)

In [ ]:
max_lr=1e-2
momentum=0.9
weight_decay = 1e-4
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_test.parameters(), lr=max_lr, momentum=momentum, weight_decay=weight_decay)

In [ ]:
print([param.requires_grad for param in model_test.parameters()])

In [ ]:
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, min_lr=5e-05)

In [ ]:
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.3)

In [ ]:
 scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[40,55,60,80,120,140], gamma=0.1)

In [ ]:
train_loss=[]
train_acc=[]
test_loss=[]
test_acc=[]
num_epochs=100
for epoch in range(num_epochs):
    model_test.train()
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    data_loader = tqdm(test_dl, total=len(test_dl), desc=f'Epoch [{epoch + 1}/{num_epochs}]')

    for batch_idx, (inputs, targets) in enumerate(data_loader):
        optimizer.zero_grad()
        inputs, targets = inputs.to(device), targets.to(device)


        outputs = model_test(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        _, predicted = outputs.max(1)
        correct_predictions += predicted.eq(targets).sum().item()
        total_samples += targets.size(0)

        # Progress bar description
        data_loader.set_postfix(loss=total_loss / (batch_idx + 1), accuracy=correct_predictions / total_samples)

    average_loss = total_loss / len(test_dl)
    accuracy = correct_predictions / total_samples
    
    train_loss.append(average_loss)
    train_acc.append(accuracy)
    
    model_test.eval()
    # test the model
    correct_test, total_test, total_loss_test = 0, 0, 0
    with torch.no_grad():
        for (inputs, targets) in test_dl:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model_test(inputs)
            loss = criterion(outputs, targets)
            total_loss_test += loss.item()

            __, predicted = outputs.max(1)
            correct_test += predicted.eq(targets).sum().item()
            total_test+= targets.size(0)

        accuracy_test = correct_test / total_test
        average_loss_test = total_loss_test / len(test_dl)
    
    test_loss.append(average_loss_test)
    test_acc.append(accuracy_test)
    
    print(f'Epoch [{epoch + 1}/{num_epochs}] - Loss: {average_loss:.4f}, Accuracy: {accuracy * 100:.2f}%, Loss on test data: {average_loss_test:.4f} Accuracy on test data: {100 * accuracy_test:.2f} %')

    model_save_path = '/kaggle/working/aftermodels/aftertl_100cl_ep' + str(epoch+1) + '.pth'
    training_stats = {
      'epoch': epoch+1,
      'model_state_dict': model_test.state_dict(),
      'optimizer_state_dict': optimizer.state_dict(),
      'scheduler_state_dict': scheduler.state_dict(),
      'loss': average_loss,
      'accuracy': accuracy,
      'test_accuracy': accuracy_test,
      'test_loss': average_loss_test,
      'learning_rate': optimizer.param_groups[0]["lr"]
    }
        
    scheduler.step()
    torch.save(training_stats,model_save_path)

# Plots

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Training and Testing Accuracy")
plt.plot(test_acc,label="Testing accuracy")
plt.plot(train_acc,label="Training accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.grid()
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Training and Testing Loss")
plt.plot(test_loss,label="Testing loss")
plt.plot(train_loss,label="Training loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
model_test.eval()
ep=3
for e in range(ep):
    correct_test, total_test= 0, 0
    with torch.no_grad():
        for (inputs, targets) in test_dl:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model_test(inputs)
            __, predicted = outputs.max(1)
            correct_test += predicted.eq(targets).sum().item()
            total_test+= targets.size(0)

    accuracy_test = correct_test / total_test
    accuracy_test_prc=accuracy_test*100
    print(f'Accuracy on test data: {accuracy_test_prc:.2f} %')

In [ ]:
model_save_path = '/kaggle/working/resnet_cifar100_transferlearningfor100clasep14' + str(epoch+1) + '.pth'

In [ ]:
training_stats = {
  'epoch': epoch+1,
  'model_state_dict': model_test.state_dict(),
  'optimizer_state_dict': optimizer.state_dict(),
  'scheduler_state_dict': scheduler.state_dict(),
  'loss': average_loss,
  'accuracy': accuracy,
  'test_accuracy': accuracy_test,
  'test_loss': average_loss_test,
  'learning_rate': optimizer.param_groups[0]["lr"]
}

In [ ]:
torch.save(training_stats, model_save_path)